<a target="_blank" href="https://colab.research.google.com/github/gox6/colab-demos/blob/main/rags/evaluate-rags-rigorously-or-perish.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [13]:
# Installing Python packages & hiding
!pip install langchain ragas xmltodict &> /dev/null

In [26]:
# Managing secrets
# - If using Colab please use Colab Secrets
# - If running outside Colab please provide secrets as ENV VARS

import sys
COLAB = 'google.colab' in sys.modules

if COLAB:
  from google.colab import userdata
  OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
else:
  import os
  OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')



In [20]:
from langchain.document_loaders import PubMedLoader

loader = PubMedLoader("liver", load_max_docs=10)
documents = loader.load()

In [25]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# documents = load your documents

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k", API_KEY=OPENAI_API_KEY)
critic_llm = ChatOpenAI(model="gpt-4", API_KEY=OPENAI_API_KEY)
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(documents, 10, distributions)
testset.to_pandas()

embedding nodes:   0%|          | 0/20 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What were the objectives and findings of the R...,[BACKGROUND AND AIM: The REgistry of Selective...,The objectives of the RESIN study were to eval...,simple,"[{'uid': '38615197', 'Title': 'Multicenter eva...",True
1,What is the relationship between the farnesoid...,[BACKGROUND AND AIM: Small heterodimer partner...,The relationship between the farnesoid X recep...,simple,"[{'uid': '38615196', 'Title': 'The contributio...",True
2,How do thermotropic drugs enhance the cryodest...,[Cryoablation has become a valuable treatment ...,Thermotropic drugs enhance the cryodestruction...,simple,"[{'uid': '38615077', 'Title': 'Cryoablation wi...",True
3,What are the effects of COVID-19 infection on ...,[The study intended to determine the correlati...,COVID-19 infection can induce disturbances in ...,simple,"[{'uid': '38615200', 'Title': 'Identification ...",True
4,"What are the demographics, survival rates, and...",[INTRODUCTION: Pancreatic ductal adenocarcinom...,The demographics of pancreatic ductal adenocar...,simple,"[{'uid': '38615292', 'Title': 'Pancreatic duct...",True
5,What is the impact of multimodal therapy on PD...,[INTRODUCTION: Pancreatic ductal adenocarcinom...,nan,multi_context,"[{'uid': '38615292', 'Title': 'Pancreatic duct...",True
6,"What is the relationship between SHP knockout,...",[BACKGROUND AND AIM: Small heterodimer partner...,nan,multi_context,"[{'uid': '38615196', 'Title': 'The contributio...",True
7,What is a major challenge in cryosurgery and h...,[Cryoablation has become a valuable treatment ...,One of the major challenges in cryosurgery is ...,multi_context,"[{'uid': '38615077', 'Title': 'Cryoablation wi...",True
8,What is a major challenge in cryosurgery and h...,[Cryoablation has become a valuable treatment ...,One of the major challenges in cryosurgery is ...,multi_context,"[{'uid': '38615077', 'Title': 'Cryoablation wi...",True
9,What are the top predictors of high-risk MASH ...,[Early identification of high-risk metabolic d...,"ALT, GGT, platelet count, waist circumference,...",reasoning,"[{'uid': '38615137', 'Title': 'An explainable ...",True
